In [ ]:
import pandas as pd
import os
import subprocess

praat_path = r".exe" 

# Directory containing audio and TextGrid files
data_dir = r" "

praat_script = r".praat"

excel_file = r"manID.xlsx"  

# Read IDs from Excel column
ids_df = pd.read_excel(excel_file, header=None, names=['ID'])

In [ ]:
import os
import re
import pandas as pd
import subprocess

# Initialize DataFrame for results
results_f1_df = pd.DataFrame(columns=['ID', 'loudness'])

def clean_illegal_characters(s):
    """Remove illegal characters from a string."""
    return re.sub(r'[\x00-\x1F\x7F]', '', s)

# Assuming ids_df is defined and contains the 'ID' column with relevant IDs
for id in ids_df['ID']:
    audio_paths = []
    textgrid_paths = []

    # Iterate over files in the directory
    for filename in os.listdir(data_dir):
        if filename.startswith(id) and filename.endswith(".wav"):
            audio_paths.append(os.path.join(data_dir, filename))
        elif filename.startswith(id) and filename.endswith(".TextGrid"):
            textgrid_paths.append(os.path.join(data_dir, filename))

    # Process all versions of the ID
    for audio_path, textgrid_path in zip(audio_paths, textgrid_paths):
        result = subprocess.run([praat_path, praat_script, audio_path, textgrid_path], capture_output=True, text=True)
        clean_result = clean_illegal_characters(result.stdout.strip())

        f1_match = re.search(r'L: (\d+\.?\d*)', clean_result)
        if f1_match:
            f1_value = float(f1_match.group(1))
            # Append to DataFrame
            results_f1_df = pd.concat([results_f1_df, pd.DataFrame([{'ID': id, 'loudness': f1_value}])], ignore_index=True)

# Save results to CSV

results_f1_df.to_csv("loudness.csv", index=False)
    